In [ ]:
def winmatch(x):
    request = mechanize.Request("http://waterrights.utah.gov/wellinfo/wellsearch.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0] 
    #print form
    form["mainoption"]=["radius"]
    form["SearchRadius"]="2000"
    form["option"]=["UTM"]
    form["xUTM"]=str(x[0])
    form["yUTM"]=str(x[1])
    if np.isfinite(x[2]):
        form["DateRange"]=["on"]
        form["edtDrillBef"]=str(int(str(x[2])[:4])+5)
        form["edtDrillAft"]=str(int(str(x[2])[:4])-5)
        #print str(int(str(x[2])[:4]))
    if np.isfinite(x[3]):
        form["DepthRange"]=["on"]
        form["edtDrillBel"]=str(int(x[3])-10)
        form["edtDrillAbo"]=str(int(x[3])+30)
        #print str(str(int(x[3])))
    win =  mechanize.urlopen(form.click()).read()
    winbeg = win.find('WIN=')
    if winbeg == -1:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan 
    else:
        wintabeg=win.find('<table',win.find('<table')+5)
        wintaend=win.find('</table>')
        winmatches = pd.read_html(win[wintabeg:wintaend], header=0, skiprows=0)
        winmatches = winmatches[0]
        winDic = {u'WRNUM/Appl. No.':'WRNUM',u'Distance From Point (ft)':'DIST',u'Diameter':'Diam',u'Depth':'TD',
          u'Drilled Date':'DrillDate',u'Location(link to Log)':'Locatio',u'WIN':'WIN',u'Geologic Log':'Log'}
        winmatches.rename(columns=winDic,inplace=True)
        return winmatches.ix[0,0], winmatches.ix[0,1], winmatches.ix[0,2], winmatches.ix[0,3], winmatches.ix[0,4], winmatches.ix[0,5], int(winmatches.ix[0,6])

winDic = {u'WRNUM/Appl. No.':'WRNUM',u'Distance From Point (ft)':'DIST',u'Diameter':'Diam',u'Depth':'TD',
          u'Drilled Date':'DrillDate',u'Location(link to Log)':'Locatio',u'WIN':'WIN',u'Geologic Log':'Log'}
winmatches.rename(columns=winDic,inplace=True)
list(winmatches.ix[0])

http://wwwsearch.sourceforge.net/mechanize/forms.html

In [1]:
def getwellinfo(x):
    request = mechanize.Request("http://maps.waterrights.utah.gov/asp/location.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0]
    form["UTMx"]= str(x[0])
    form["UTMy"]= str(x[1])
    form["datumutm"]=["NAD83"]
    desc =  mechanize.urlopen(form.click()).read()
    try:
        PLSS, CAD = getPLSS(desc)
    except(ValueError):
        PLSS, CAD = np.nan, np.nan
    return PLSS, CAD

def getwellPLSS(x):
    request = mechanize.Request("http://maps.waterrights.utah.gov/asp/location.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0]
    form["UTMx"]= str(x[0])
    form["UTMy"]= str(x[1])
    form["datumutm"]=["NAD83"]
    desc =  mechanize.urlopen(form.click()).read()
    try:
        PLSS, CAD = getPLSS(desc)
    except(ValueError):
        PLSS, CAD = np.nan, np.nan
    return PLSS

def getwellCAD(x):
    request = mechanize.Request("http://maps.waterrights.utah.gov/asp/location.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0]
    form["UTMx"]= str(x[0])
    form["UTMy"]= str(x[1])
    form["datumutm"]=["NAD83"]
    desc =  mechanize.urlopen(form.click()).read()
    try:
        PLSS, CAD = getPLSS(desc)
    except(ValueError):
        PLSS, CAD = np.nan, np.nan
    return PLSS

In [ ]:
def getPLSS(desc):
    PLSStitlen = len("PLSS Description is </font><font size='4'> ")
    PLSSbeg = desc.find("PLSS Description is </font><font size='4'>")+PLSStitlen
    PLSSend = desc[PLSSbeg:].find(" <br> ")+PLSSbeg
    PLSSdesc = desc[PLSSbeg:PLSSend]
    QRTRend = desc.find(" of the above Section </font></center><p>")
    QRTRbeg = desc.find("The point is found in the ")+len("The point is found in the ")
    QRTRdesc = desc[QRTRbeg:QRTRend]
    SecBeg = PLSSdesc.find("Section ")+len("Section ")
    SecEnd = PLSSdesc.find(", Township ")
    Section = int(PLSSdesc[SecBeg:SecEnd])
    TownBeg = SecEnd + len(", Township ")
    TownEnd = PLSSdesc.find(", Range ")
    Township = PLSSdesc[TownBeg:TownEnd]
    RangeBeg = TownEnd + len(", Range ")
    RangeEnd = RangeBeg + PLSSdesc[RangeBeg:].find(", ")
    Range = PLSSdesc[RangeBeg:RangeEnd]

    PLSS = PLSSdesc.replace("South ","S")
    PLSS = PLSS.replace("West ","W")
    PLSS = PLSS.replace("North ","N")
    PLSS = PLSS.replace("East ","E")
    PLSS = PLSS.replace("feet ","")
    PLSS = PLSS.replace(" and "," ")
    PLSS = PLSS.replace("from the ","")
    BM = PLSS[-6:-4]
    PLSScn = PLSS.find(" Corner")
    PLSS = PLSS[:PLSScn]
    PLSS = PLSS.replace(',','')
    PLSS = PLSS + " " +str(Section) + " " + Township + " " + Range + " " + BM
    
    try:
        CAD = qqq(PLSS)
    except(AttributeError,TypeError,UnboundLocalError):
        #print PLSS
        CAD = np.nan
    return PLSS, CAD

In [ ]:
def qqq(x):
    x.rstrip().lstrip()
    j = x.split(' ')
    a = j[0][:1]
    b = j[0][1:]
    c = j[1][:1]
    d = j[1][1:]
    e = [a,b,c,d,j[2],j[3],j[4],j[5],j[6]]
    
    NS = int(e[1].replace(',',''))
    EW = int(e[3].replace(',',''))
    qc = e[4]
    d1 = e[0]
    d2 = e[2]
    dic1 = {'NE':'a','NW':'b','SW':'c','SE':'d'}
    qcdDict = {'E4S':'d','E4N':'a','N4E':'a','N4W':'b','W4N':'b','W4S':'c','S4W':'c','S4E':'d'}
    dic2 = {'a':'b','b':'a','c':'d','d':'c'}
    dic3 = {'a':'d','b':'c','c':'b','d':'a'}
    dic4 = {'a':'c','b':'d','c':'a','d':'b'}
    if qc[-1]=='4':
        if qc[0]=='N' or qc[0]=='S':
            qcd = qc+d2
        elif qc[0]=='E' or qc[0]=='W':
            qcd = qc+d1
        q1 = qcdDict.get(qcd,'x')
    elif qc in ('NE','NW','SW','SE'):
        q1 = dic1.get(qc)
    else:
        print "invalid quarter"
        q1 = 'X'
    if NS < 1320:
        if EW <1320:
            q2 = q1
        elif EW >1320:
            qd2 = {'a':'b','b':'a','c':'d','d':'c'}
            q2 = dic2.get(q1,'x')
    elif NS > 1320:
        if EW <1320:
            q2 = dic3.get(q1,'x')
        elif EW >1320:
            q2 = dic4.get(q1,'x')
    else:
        q2 = 'X'

    if NS < 660 or (NS > 1320 and NS < 1980):
        if (EW < 660) or (EW > 1320 and EW < 1980):
            q3 = q1
        elif (EW > 660 and EW < 1320) or (EW > 1980 and EW < 2640):
            q3 = dic2.get(q1,'x')
    elif (NS > 660 and NS < 1320) or (NS > 1980 and NS < 2640):
        if (EW < 660) or (EW > 1320 and EW < 1980):
            q3 = dic3.get(q1,'x')
        elif (EW > 660 and EW < 1320) or (EW > 1980 and EW < 2640):
            q3 = dic4.get(q1,'x')
    else:
        q3 = 'X'
    Tn = e[6][:-1].rjust(2)
    Rn = e[7][:-1].rjust(2)
    Sec = e[5].rjust(2)
    TRd = e[6][-1]+e[7][-1]
    TR = dic1.get(TRd).upper()
    CAD = '('+TR+'-'+Tn+'-'+Rn+')'+Sec+q1+q2+q3+'-1'
    return CAD  